## Pre-requisites: Setting up path and dependencies

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/MN-20-Credit/Aerial-YOLO-DOTA/src

/content/drive/MyDrive/MN-20-Credit/Aerial-YOLO-DOTA/src


In [ ]:
!pip install ultralytics pyyaml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 32.6 MB/s eta 0:00:00


##Attack Configs

In [ ]:
import argparse
import json
import os
import cv2
from tqdm import tqdm
import numpy as np
from ultralytics import YOLO
from helpers import read_yaml_config
import torchvision.transforms as T
import torch

model_path = "/content/drive/MyDrive/MN-20-Credit/models/yolov9/best.pt"
model = YOLO(model_path)

img_path = "/content/drive/MyDrive/MN-20-Credit/image/P0017.png"
img_name = "P0017.png"
white_img_name = "WhiteP0017.png"

config = read_yaml_config("dota.yaml")
class_names = config['names']
data_root = config["path"]

# Directory to save annotated images
annotated_dir = os.path.join(data_root, "annotated_test_images")
os.makedirs(annotated_dir, exist_ok=True)

output_path = img_name

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


### Preprocess the image

In [ ]:
def _select_device(device):
    if device is None:
        return "cuda" if torch.cuda.is_available() else "cpu"
    return device

In [ ]:
def _read_image_bgr(image_path):
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Unable to read image at {image_path}")
    return img  # BGR

def _bgr_to_rgb(img_bgr):
    return cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

In [ ]:
def _make_transform(size=(640, 640)):
    return T.Compose([
        T.ToPILImage(),
        T.Resize(size),
        T.ToTensor(),   # converts to [0,1], CxHxW
    ])

In [ ]:
def _preprocess_to_tensor(img_bgr, device, size=(640, 640)):
    """
    Reads BGR img, converts to RGB, applies transforms and returns tensor on device
    with shape (1,3,H,W) and requires_grad=True.
    """
    img_rgb = _bgr_to_rgb(img_bgr)
    transform = _make_transform(size)
    tensor = transform(img_rgb).unsqueeze(0).to(device)  # (1,3,H,W)
    tensor.requires_grad_(True)
    return tensor

### Forward pass to get the model output

In [ ]:
def _forward_with_grad(model, input_tensor):
    """
    Runs model forward ensuring gradients are enabled for input_tensor.
    Returns the raw model outputs.
    """
    model.eval()
    with torch.set_grad_enabled(True):
        # outputs = model(input_tensor)
        preds = model.model(input_tensor)  # call underlying nn.Module
    return preds
    # return outputs

### Compute the loss and backward gradient

In [ ]:
def _compute_confidence_loss(preds, device):
  """
    Define a loss directly from raw YOLO predictions.
    preds is a list of predictions (tensor) from model.model.
  """
  # preds[0] shape: (batch, anchors, (x,y,w,h,obj,cls...))
  raw = preds[0]
  # take objectness logits (before sigmoid)
  obj_logits = raw[..., 4]  # this is differentiable
  # clone to avoid inference-tensor restriction
  obj_logits = obj_logits.clone()
  # loss: negative mean objectness (so attack reduces detection confidence)
  return -obj_logits.mean()

In [ ]:
def _backward_on_loss(model, loss):
    """
    Zeroes model grads and calls backward on loss if non-zero (like original).
    """
    model.zero_grad()
    if loss.item() != 0.0:
        loss.backward()



```
# This is formatted as code
```

### Define FGSM formula

In [ ]:
def _apply_fgsm(image_tensor, eps):
    """
    Applies FGSM using sign of image_tensor.grad and clamps to [0,1].
    If no grad, returns original tensor (detached).
    """
    if image_tensor.grad is None:
        return image_tensor.detach()
    adv = image_tensor + eps * image_tensor.grad.sign()
    adv = torch.clamp(adv, 0.0, 1.0)
    return adv.detach()

In [ ]:
def _save_adv_tensor_as_png(adv_tensor, save_path):
    """
    Expects adv_tensor on CPU or GPU with shape (1,3,H,W), values in [0,1].
    Converts to uint8 RGB -> BGR and writes with OpenCV.
    """
    arr = adv_tensor.detach().cpu().squeeze(0).permute(1, 2, 0).numpy()  # H,W,3 RGB
    uint8 = (arr * 255.0).round().astype('uint8')
    bgr = uint8[:, :, ::-1]
    cv2.imwrite(save_path, bgr)
    return save_path

### Implement the attack by calling helper functions

In [ ]:
def whitebox_fgsm_attack(model, image_path, eps=0.02, device=None, save_path=white_img_name):
    """
    Refactored: same behaviour as your original function but split into small helpers.
    Returns: (adv_image_tensor, adv_results)
    """
    device = _select_device(device)

    # Read image (BGR) and preprocess to tensor with grad enabled
    image = _read_image_bgr(image_path)
    image_tensor = _preprocess_to_tensor(image, device, size=(640, 640))

    # Ensure model on device and evaluation mode (same as original)
    model.to(device)
    model.eval()

    # Forward pass with gradients enabled for input
    outputs = _forward_with_grad(model, image_tensor)

    # Compute loss (negative summed confidences) exactly like your original
    loss = _compute_confidence_loss(outputs, device)

    # Backward to populate image_tensor.grad
    _backward_on_loss(model, loss)

    # FGSM perturbation
    adv_image = _apply_fgsm(image_tensor, eps)

    # Save adversarial image (convert to uint8 and BGR)
    _save_adv_tensor_as_png(adv_image, save_path)
    print(f"Saved adversarial image to {save_path}")

    # Return adversarial tensor and model outputs on adversarial image (same final call as original)
    return adv_image, model(adv_image)

# --- call the function (keeps your original final call) ---
whitebox_fgsm_attack(model, img_path, eps=0.02, device=None, save_path=white_img_name)



Saved adversarial image to WhiteP0017.png

0: 640x640 26 planes, 3 small-vehicles, 75.7ms
Speed: 0.1ms preprocess, 75.7ms inference, 271.4ms postprocess per image at shape (1, 3, 640, 640)


(tensor([[[[0.0000, 0.0000, 0.0200,  ..., 0.0000, 0.0000, 0.0200],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0200, 0.0200],
           [0.0000, 0.0200, 0.0000,  ..., 0.0000, 0.0000, 0.0200],
           ...,
           [0.0200, 0.0000, 0.0000,  ..., 0.0200, 0.0000, 0.0000],
           [0.0200, 0.0200, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0200, 0.0200, 0.0200,  ..., 0.0000, 0.0200, 0.0000]],
 
          [[0.0200, 0.0200, 0.0200,  ..., 0.0200, 0.0000, 0.0000],
           [0.0200, 0.0200, 0.0200,  ..., 0.0000, 0.0000, 0.0200],
           [0.0000, 0.0200, 0.0000,  ..., 0.0000, 0.0200, 0.0200],
           ...,
           [0.0200, 0.0200, 0.0000,  ..., 0.0200, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0200, 0.0200, 0.0000]],
 
          [[0.0200, 0.0200, 0.0000,  ..., 0.0200, 0.0000, 0.0000],
           [0.0200, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.00

## Generating Adversarial images on Val set

In [ ]:
import os
import json
from tqdm import tqdm
import numpy as np
import cv2
import yaml

# === helper: load YOLO-format GT label -> xyxy pixel coords ===
def load_gt_boxes_from_yolo(label_path, img_shape):
    """
    label_path: path to YOLO .txt labels for one image (class x_center y_center w h) normalized
    img_shape: (H, W, C)
    returns: list of {"bbox":[x1,y1,x2,y2], "class": int}
    """
    h, w = img_shape[:2]
    gt = []
    if not os.path.exists(label_path):
        return gt
    with open(label_path, "r") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) < 5:
                continue
            cls = int(float(parts[0]))
            xc, yc, bw, bh = map(float, parts[1:5])
            x1 = (xc - bw/2) * w
            y1 = (yc - bh/2) * h
            x2 = (xc + bw/2) * w
            y2 = (yc + bh/2) * h
            gt.append({"bbox":[x1,y1,x2,y2], "class": cls})
    return gt

# === helper: extract boxes from ultralytics Results object ===
def extract_boxes_from_results(results):
    """
    results: model(image) or model(path) return for single image
    returns: boxes (Nx4 array), classes (N), confs (N)
    If no boxes, returns empty arrays.
    """
    if not results or results[0] is None or getattr(results[0], "boxes", None) is None:
        return np.zeros((0,4)), np.zeros((0,)), np.zeros((0,))
    boxes = results[0].boxes.xyxy.cpu().numpy()
    confs = results[0].boxes.conf.cpu().numpy()
    clss  = results[0].boxes.cls.cpu().numpy()
    return boxes, clss, confs

# === IoU for matching ===
def bbox_iou(box1, box2):
    x1 = max(box1[0], box2[0]); y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2]); y2 = min(box1[3], box2[3])
    inter_w = max(0.0, x2 - x1); inter_h = max(0.0, y2 - y1)
    inter = inter_w * inter_h
    area1 = max(0.0, (box1[2]-box1[0])*(box1[3]-box1[1]))
    area2 = max(0.0, (box2[2]-box2[0])*(box2[3]-box2[1]))
    union = area1 + area2 - inter
    return inter/union if union>0 else 0.0

# === save detection list to JSON helper ===
def save_detections_json(boxes, clss, confs, class_names, outpath):
    out = []
    for i,(b,c,p) in enumerate(zip(boxes, clss, confs)):
        out.append({
            "class_id": int(c),
            "class": class_names[int(c)] if isinstance(class_names, (list,tuple,dict)) else int(c),
            "confidence": float(round(float(p), 4)),
            "bbox": [float(round(float(x),3)) for x in b.tolist()]
        })
    with open(outpath, "w") as f:
        json.dump(out, f, indent=2)
    return outpath

# === Main runner that attacks all images in a folder and computes metrics ===
def run_attack_on_folder_and_evaluate(model,
                                      val_images_dir,
                                      val_labels_dir,
                                      out_adv_dir,
                                      class_names,
                                      eps=0.02,
                                      conf_thresh=0.001,
                                      iou_match_thresh=0.5,
                                      device=None,
                                      write_val_yaml=False,
                                      attacked_yaml_out="/content/attacked_tmp.yaml"):
    """
    - val_images_dir: folder with original val images (e.g. data/images/val)
    - val_labels_dir: folder with corresponding YOLO labels (e.g. data/labels/val)
    - out_adv_dir: output directory where adversarial images will be written
    - class_names: list/dict of class names
    - eps: FGSM epsilon in [0,1]
    - returns aggregated metrics dict
    """
    os.makedirs(out_adv_dir, exist_ok=True)
    os.makedirs(os.path.join(out_adv_dir, "json"), exist_ok=True)
    os.makedirs(os.path.join(out_adv_dir, "orig_json"), exist_ok=True)

    image_files = sorted([f for f in os.listdir(val_images_dir) if f.lower().endswith(('.jpg','.jpeg','.png'))])
    if len(image_files)==0:
        raise RuntimeError(f"No images found in {val_images_dir}")

    # accumulators
    total_orig_dets = 0
    total_matched = 0
    total_unmatched_orig = 0
    total_unmatched_adv = 0
    conf_drop_list = []

    for fname in tqdm(image_files, desc="Attacking images"):
        orig_path = os.path.join(val_images_dir, fname)
        label_path = os.path.join(val_labels_dir, os.path.splitext(fname)[0] + ".txt")
        adv_path = os.path.join(out_adv_dir, fname)

        # 1) run whitebox attack and save adv image
        # expects whitebox_fgsm_attack to save adv image at save_path and to return (adv_tensor, adv_results)
        try:
            # you may pass device or leave None
            adv_tensor, adv_results = whitebox_fgsm_attack(model, orig_path, eps=eps, device=device, save_path=adv_path)
        except Exception as e:
            # If your whitebox function raises, fallback: try calling it but still continue
            print(f"[WARN] whitebox_fgsm_attack failed for {fname}: {e}")
            continue

        # 2) run detection on original image and adversarial image (if adv_results is None or not usable)
        results_orig = model(orig_path)   # Returns Results
        if adv_results is None:
            results_adv = model(adv_path)
        else:
            # adv_results might be a Results object already (if your function returned it)
            # If it is a tensor, call model on adv_path
            try:
                # If adv_results looks like Ultralytics Results, use it; else run inference
                if isinstance(adv_results, type(results_orig)):
                    results_adv = adv_results
                else:
                    results_adv = model(adv_path)
            except Exception:
                results_adv = model(adv_path)

        # 3) extract boxes
        boxes_o, clss_o, confs_o = extract_boxes_from_results(results_orig)
        boxes_a, clss_a, confs_a = extract_boxes_from_results(results_adv)

        # Save per-image detection JSONs
        save_detections_json(boxes_o, clss_o, confs_o, class_names, os.path.join(out_adv_dir, "orig_json", os.path.splitext(fname)[0]+".json"))
        save_detections_json(boxes_a, clss_a, confs_a, class_names, os.path.join(out_adv_dir, "json", os.path.splitext(fname)[0]+".json"))

        # 4) compute matches (greedy per-original detection)
        total_orig_dets += len(boxes_o)
        used_adv = set()
        for i, (bo, co) in enumerate(zip(boxes_o, clss_o)):
            best_j = -1; best_iou = 0.0
            for j, (ba, ca) in enumerate(zip(boxes_a, clss_a)):
                if j in used_adv or int(co) != int(ca):
                    continue
                iou = bbox_iou(bo, ba)
                if iou > best_iou:
                    best_iou = iou; best_j = j
            if best_j >= 0 and best_iou >= iou_match_thresh:
                total_matched += 1
                used_adv.add(best_j)
                conf_drop_list.append(float(confs_o[i]) - float(confs_a[best_j]))
            else:
                total_unmatched_orig += 1
        # any adv boxes not used are new false positives
        total_unmatched_adv += (len(boxes_a) - len(used_adv))

    # Aggregate metrics
    detection_drop_rate = total_unmatched_orig / total_orig_dets if total_orig_dets>0 else 0.0
    avg_conf_drop = float(np.mean(conf_drop_list)) if len(conf_drop_list)>0 else 0.0

    metrics = {
        "total_images": len(image_files),
        "total_orig_detections": int(total_orig_dets),
        "total_matched": int(total_matched),
        "total_unmatched_orig": int(total_unmatched_orig),
        "total_unmatched_adv": int(total_unmatched_adv),
        "detection_drop_rate": float(detection_drop_rate),
        "avg_confidence_drop": float(avg_conf_drop),
    }

    print("\n=== ATTACK AGGREGATE METRICS ===")
    print(json.dumps(metrics, indent=2))

    # Optional: write a tiny dataset YAML pointing to attacked images so you can call model.val() on them
    if write_val_yaml:
        root = os.path.abspath(out_adv_dir)
        data = {
            "path": root,
            "train": "images/train",
            "val": "images",   # our attacked images are directly in out_adv_dir (set to images if needed)
            "names": {i:n for i,n in enumerate(class_names)},
            "nc": len(class_names)
        }
        with open(attacked_yaml_out, "w") as f:
            yaml.safe_dump(data, f, sort_keys=False)
        print(f"Wrote attacked dataset yaml to {attacked_yaml_out} (use model.val(data=...))")

    return metrics

# ============ Example usage ============
if __name__ == "__main__":
    # Paths: replace these with your actual paths
    VAL_IMAGES = "/content/drive/MyDrive/MN-20-Credit/dota-yolo/images/val"        # original validation images
    VAL_LABELS = "/content/drive/MyDrive/MN-20-Credit/dota-yolo/labels/val"     # YOLO labels (.txt)
    OUT_ADV_DIR = "/content/drive/MyDrive/MN-20-Credit/dota-yolo/images_white_adv/val"  # where adv images will be written
    EPS = 0.02
    DEVICE = "cuda" if __import__("torch").cuda.is_available() else "cpu"

    # class_names loaded earlier from your dota.yaml, or supply a list
    # config = read_yaml_config("dota.yaml")
    # class_names = config['names']
    # For example:
    # class_names = ["plane","vehicle",...]

    # run
    metrics = run_attack_on_folder_and_evaluate(
        model=model,
        val_images_dir=VAL_IMAGES,
        val_labels_dir=VAL_LABELS,
        out_adv_dir=OUT_ADV_DIR,
        class_names=class_names,
        eps=EPS,
        conf_thresh=0.001,
        iou_match_thresh=0.5,
        device=DEVICE,
        write_val_yaml=True,              # write YAML if you want to call model.val() manually later
        attacked_yaml_out="/content/attacked_tmp.yaml"
    )

    # Optional: call model.val() on attacked set (uncomment if you want mAP)
    # metrics_attack = model.val(data="/content/attacked_tmp.yaml", split="val", imgsz=640, conf=0.001, iou=0.5, device=DEVICE)
    # print("mAP for attacked set:", metrics_attack.box.map50, metrics_attack.box.map)


Attacking images:   0%|          | 0/374 [00:00<?, ?it/s]

Saved adversarial image to /content/drive/MyDrive/MN-20-Credit/dota-yolo/images_white_adv/val/P0003.png

0: 640x640 38 large-vehicles, 14 small-vehicles, 6803.6ms
Speed: 0.1ms preprocess, 6803.6ms inference, 9.1ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/drive/MyDrive/MN-20-Credit/dota-yolo/images/val/P0003.png: 576x640 40 large-vehicles, 15 small-vehicles, 4504.8ms
Speed: 15.4ms preprocess, 4504.8ms inference, 1.4ms postprocess per image at shape (1, 3, 576, 640)


Attacking images:   0%|          | 1/374 [00:24<2:33:37, 24.71s/it]

Saved adversarial image to /content/drive/MyDrive/MN-20-Credit/dota-yolo/images_white_adv/val/P0011.png

0: 640x640 57 large-vehicles, 1 ship, 35 small-vehicles, 4913.2ms
Speed: 0.0ms preprocess, 4913.2ms inference, 8.1ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/drive/MyDrive/MN-20-Credit/dota-yolo/images/val/P0011.png: 640x576 56 large-vehicles, 1 ship, 35 small-vehicles, 5828.3ms
Speed: 4.3ms preprocess, 5828.3ms inference, 2.6ms postprocess per image at shape (1, 3, 640, 576)


Attacking images:   1%|          | 2/374 [00:48<2:31:35, 24.45s/it]

...

Saved adversarial image to /content/drive/MyDrive/MN-20-Credit/dota-yolo/images_white_adv/val/P2797.png

0: 640x640 4 large-vehicles, 3 planes, 4900.6ms
Speed: 0.0ms preprocess, 4900.6ms inference, 7.2ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/drive/MyDrive/MN-20-Credit/dota-yolo/images/val/P2797.png: 640x320 1 large-vehicle, 4 planes, 7 small-vehicles, 2967.2ms
Speed: 3.0ms preprocess, 2967.2ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 320)


Attacking images: 100%|█████████▉| 373/374 [2:24:42<00:22, 22.19s/it]

Saved adversarial image to /content/drive/MyDrive/MN-20-Credit/dota-yolo/images_white_adv/val/P2804.png

0: 640x640 6 large-vehicles, 33 planes, 3 small-vehicles, 6774.8ms
Speed: 0.0ms preprocess, 6774.8ms inference, 5.8ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/drive/MyDrive/MN-20-Credit/dota-yolo/images/val/P2804.png: 416x640 3 large-vehicles, 26 planes, 132 small-vehicles, 3186.3ms
Speed: 3.8ms preprocess, 3186.3ms inference, 1.4ms postprocess per image at shape (1, 3, 416, 640)


Attacking images: 100%|██████████| 374/374 [2:25:05<00:00, 23.28s/it]


=== ATTACK AGGREGATE METRICS ===
{
  "total_images": 374,
  "total_orig_detections": 23372,
  "total_matched": 78,
  "total_unmatched_orig": 23294,
  "total_unmatched_adv": 18372,
  "detection_drop_rate": 0.9966626732842718,
  "avg_confidence_drop": 0.011144480262047205
}
Wrote attacked dataset yaml to /content/attacked_tmp.yaml (use model.val(data=...))


## Metric Analysis

In [ ]:
import yaml, os

base = "/content/drive/MyDrive/MN-20-Credit/dota-yolo/whitebox"
ATTACK_YAML = "/content/attacked_val.yaml"

data_dict = {
    "path": base,
    "train": os.path.join(base, "images/val"),
    "val": os.path.join(base, "images/val"),
    "nc": len(class_names),
    "names": {i: n for i, n in enumerate(class_names)},
    # 👇 Explicit label path override
    "labels": os.path.join(base, "labels/val")
}

with open(ATTACK_YAML, "w") as f:
    yaml.safe_dump(data_dict, f, sort_keys=False)

print(open(ATTACK_YAML).read())


path: /content/drive/MyDrive/MN-20-Credit/dota-yolo/whitebox
train: /content/drive/MyDrive/MN-20-Credit/dota-yolo/whitebox/images/val
val: /content/drive/MyDrive/MN-20-Credit/dota-yolo/whitebox/images/val
nc: 16
names:
  0: 0
  1: 1
  2: 2
  3: 3
  4: 4
  5: 5
  6: 6
  7: 7
  8: 8
  9: 9
  10: 10
  11: 11
  12: 12
  13: 13
  14: 14
  15: 15
labels: /content/drive/MyDrive/MN-20-Credit/dota-yolo/whitebox/labels/val



In [ ]:
!rm /content/drive/MyDrive/MN-20-Credit/dota-yolo/images_white_adv/val.cache


In [ ]:
metrics_attack = model.val(
    data=ATTACK_YAML,
    imgsz=640,
    batch=4,              # small batch for speed
    split="val",
    device=0 if torch.cuda.is_available() else "cpu",
    half=False,
    verbose=False,
    plots=False,
    save_json=False,
)

print("\n== ATTACKED SET RESULTS ==")
print(f"mAP@0.50      : {metrics_attack.box.map50:.4f}")
print(f"mAP@0.50:0.95 : {metrics_attack.box.map:.4f}")
print(f"Precision     : {metrics_attack.box.mp:.4f}")
print(f"Recall        : {metrics_attack.box.mr:.4f}")


Ultralytics 8.3.208 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
val: Fast image access ✅ (ping: 5.3±4.6 ms, read: 36.5±23.1 MB/s, size: 746.6 KB)
val: Scanning /content/drive/MyDrive/MN-20-Credit/dota-yolo/whitebox/labels/val.cache... 374 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 374/374 445.9Kit/s 0.0s
val: /content/drive/MyDrive/MN-20-Credit/dota-yolo/whitebox/images/val/P0003.png: 55 duplicate labels removed
val: /content/drive/MyDrive/MN-20-Credit/dota-yolo/whitebox/images/val/P0011.png: 92 duplicate labels removed
val: /content/drive/MyDrive/MN-20-Credit/dota-yolo/whitebox/images/val/P0020.png: 924 duplicate labels removed
val: /content/drive/MyDrive/MN-20-Credit/dota-yolo/whitebox/images/val/P0023.png: 1663 duplicate labels removed
val: /content/drive/MyDrive/MN-20-Credit/dota-yolo/whitebox/images/val/P0027.png: 44 duplicate labels removed
val: /content/drive/MyDrive/MN-20-Credit/dota-yolo/whitebox/images/val/P0032.png: 56 duplicate labels removed
val

In [ ]:
import yaml, os

base = "/content/drive/MyDrive/MN-20-Credit/dota-yolo"
CLEAN_YAML = "/content/clean_val.yaml"

data_dict = {
    "path": base,
    "train": os.path.join(base, "images/val"),
    "val": os.path.join(base, "images/val"),
    "nc": len(class_names),
    "names": {i: n for i, n in enumerate(class_names)},
    # 👇 Explicit label path override
    "labels": os.path.join(base, "labels/val")
}

with open(CLEAN_YAML, "w") as f:
    yaml.safe_dump(data_dict, f, sort_keys=False)

print(open(CLEAN_YAML).read())

path: /content/drive/MyDrive/MN-20-Credit/dota-yolo
train: /content/drive/MyDrive/MN-20-Credit/dota-yolo/images/val
val: /content/drive/MyDrive/MN-20-Credit/dota-yolo/images/val
nc: 16
names:
  0: 0
  1: 1
  2: 2
  3: 3
  4: 4
  5: 5
  6: 6
  7: 7
  8: 8
  9: 9
  10: 10
  11: 11
  12: 12
  13: 13
  14: 14
  15: 15
labels: /content/drive/MyDrive/MN-20-Credit/dota-yolo/labels/val



In [ ]:
metrics_clean = model.val(
    data=CLEAN_YAML,  # your original val dataset
    split="val",
    imgsz=640,
    device="cuda",
    verbose=False
)

# Compare clean vs attacked
print("\n== CLEAN SET RESULTS ==")
print(f"mAP@0.50      : {metrics_clean.box.map50:.4f}")
print(f"mAP@0.50:0.95 : {metrics_clean.box.map:.4f}")
print(f"Precision     : {metrics_clean.box.mp:.4f}")
print(f"Recall        : {metrics_clean.box.mr:.4f}")

# Compute drop
print("\n== PERFORMANCE DROP ==")
print(f"Δ mAP@0.50      : {metrics_clean.box.map50 - 0.4803:.4f}")
print(f"Δ mAP@0.50:0.95 : {metrics_clean.box.map - 0.3297:.4f}")
print(f"Δ Precision     : {metrics_clean.box.mp - 0.7524:.4f}")
print(f"Δ Recall        : {metrics_clean.box.mr - 0.4104:.4f}")


Ultralytics 8.3.208 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
val: Fast image access ✅ (ping: 0.7±0.5 ms, read: 38.2±71.3 MB/s, size: 4561.2 KB)
val: Scanning /content/drive/MyDrive/MN-20-Credit/dota-yolo/labels/val... 374 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 374/374 27.6it/s 13.5s
val: /content/drive/MyDrive/MN-20-Credit/dota-yolo/images/val/P0003.png: 55 duplicate labels removed
val: /content/drive/MyDrive/MN-20-Credit/dota-yolo/images/val/P0011.png: 92 duplicate labels removed
val: /content/drive/MyDrive/MN-20-Credit/dota-yolo/images/val/P0020.png: 924 duplicate labels removed
val: /content/drive/MyDrive/MN-20-Credit/dota-yolo/images/val/P0023.png: 1663 duplicate labels removed
val: /content/drive/MyDrive/MN-20-Credit/dota-yolo/images/val/P0027.png: 44 duplicate labels removed
val: /content/drive/MyDrive/MN-20-Credit/dota-yolo/images/val/P0032.png: 56 duplicate labels removed
val: /content/drive/MyDrive/MN-20-Credit/dota-yolo/images/val/P0042.png: